# Generate mock data

In [6]:
import random
import json
import os
import numpy as np
# import tensorflow as tf
from sklearn.metrics import f1_score
import cv2

# Part 1: Generate Mock Data
data_dir = "C:/Users/zhang/PycharmProjects/ML-Coding-Practice/ScaleAI/data"
# Generating mock image names and a JSON file
num_images = 1000
class_labels = ["cat", "dog", "bird", "fish"]
mock_image = np.array(np.random.randint(0, 256, [5, 5, 3]), dtype=np.uint8)
mock_json = {
    1: ["18274", "cat"],
    2: ["15938", "dog"],
    3: ["70382", "bird"],
    4: ["28673", "fish"],
}
os.makedirs(data_dir, exist_ok=True)
# for i in range(500):
#     for class_idx in mock_json:
#         cv2.imwrite(os.path.join(data_dir, f"{mock_json[class_idx][0]}_{i:04d}.JPEG"), mock_image)
json.dump(mock_json, open("mock_classes.json", "w"))

# Step 1
Read image xxxxx_xxxx.jpeg (imageID_xxxx.jpeg). Return its class index and class label.
Input:
.json dictionary. key: class index, value: [imageID, class label]
image folder
Output:
[class index, class label]


In [7]:
dict_dir="mock_classes.json"
check_dict = json.load(open(dict_dir, "r"))
imgid_class = {}
for class_idx in check_dict:
    imgid_class[check_dict[class_idx][0]] = [class_idx, check_dict[class_idx][1]]

def get_idx_lbl(img_name):
    segments = img_name.split("_")
    imgid = segments[0]
    return imgid_class[imgid]

for item in os.listdir(data_dir):
    print(get_idx_lbl(item))

['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']
['2', 'dog']

# Step 2: Prediction
Randomly sample 500 images in the directory
Create a 4-d tensorflow tensor X
get the corresponding labels with the method provided in step 1
Shuffle the images. Predict with two models.

In [ ]:
import random
batch_size = 500
all_images = os.listdir(data_dir)
samples = random.sample(all_images, 500)
